# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fb6981db910>
├── 'a' --> tensor([[-0.7809,  0.5063,  0.5516],
│                   [-0.3210, -0.8318, -0.4972]])
└── 'x' --> <FastTreeValue 0x7fb6981db8b0>
    └── 'c' --> tensor([[ 0.1581, -0.6751,  0.1647,  1.9021],
                        [ 0.2731, -0.8836, -0.5057,  0.5117],
                        [-0.4602,  0.8365, -1.7004, -0.1574]])

In [4]:
t.a

tensor([[-0.7809,  0.5063,  0.5516],
        [-0.3210, -0.8318, -0.4972]])

In [5]:
%timeit t.a

76 ns ± 1 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fb6981db910>
├── 'a' --> tensor([[-0.5579,  0.1853,  0.4087],
│                   [-0.6326,  1.6212,  0.5685]])
└── 'x' --> <FastTreeValue 0x7fb6981db8b0>
    └── 'c' --> tensor([[ 0.1581, -0.6751,  0.1647,  1.9021],
                        [ 0.2731, -0.8836, -0.5057,  0.5117],
                        [-0.4602,  0.8365, -1.7004, -0.1574]])

In [7]:
%timeit t.a = new_value

82.7 ns ± 1.25 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.7809,  0.5063,  0.5516],
               [-0.3210, -0.8318, -0.4972]]),
    x: Batch(
           c: tensor([[ 0.1581, -0.6751,  0.1647,  1.9021],
                      [ 0.2731, -0.8836, -0.5057,  0.5117],
                      [-0.4602,  0.8365, -1.7004, -0.1574]]),
       ),
)

In [10]:
b.a

tensor([[-0.7809,  0.5063,  0.5516],
        [-0.3210, -0.8318, -0.4972]])

In [11]:
%timeit b.a

72.2 ns ± 0.787 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.5353,  1.1295, -0.0105],
               [-1.3239,  0.4582,  0.8970]]),
    x: Batch(
           c: tensor([[ 0.1581, -0.6751,  0.1647,  1.9021],
                      [ 0.2731, -0.8836, -0.5057,  0.5117],
                      [-0.4602,  0.8365, -1.7004, -0.1574]]),
       ),
)

In [13]:
%timeit b.a = new_value

609 ns ± 9.85 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

970 ns ± 10 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

13.1 µs ± 204 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

295 µs ± 10.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

263 µs ± 4.62 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fb5eaec0fd0>
├── 'a' --> tensor([[[-0.7809,  0.5063,  0.5516],
│                    [-0.3210, -0.8318, -0.4972]],
│           
│                   [[-0.7809,  0.5063,  0.5516],
│                    [-0.3210, -0.8318, -0.4972]],
│           
│                   [[-0.7809,  0.5063,  0.5516],
│                    [-0.3210, -0.8318, -0.4972]],
│           
│                   [[-0.7809,  0.5063,  0.5516],
│                    [-0.3210, -0.8318, -0.4972]],
│           
│                   [[-0.7809,  0.5063,  0.5516],
│                    [-0.3210, -0.8318, -0.4972]],
│           
│                   [[-0.7809,  0.5063,  0.5516],
│                    [-0.3210, -0.8318, -0.4972]],
│           
│                   [[-0.7809,  0.5063,  0.5516],
│                    [-0.3210, -0.8318, -0.4972]],
│           
│                   [[-0.7809,  0.5063,  0.5516],
│                    [-0.3210, -0.8318, -0.4972]]])
└── 'x' --> <FastTreeValue 0x7fb5eaec0fa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

45.3 µs ± 916 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fb69823bd90>
├── 'a' --> tensor([[-0.7809,  0.5063,  0.5516],
│                   [-0.3210, -0.8318, -0.4972],
│                   [-0.7809,  0.5063,  0.5516],
│                   [-0.3210, -0.8318, -0.4972],
│                   [-0.7809,  0.5063,  0.5516],
│                   [-0.3210, -0.8318, -0.4972],
│                   [-0.7809,  0.5063,  0.5516],
│                   [-0.3210, -0.8318, -0.4972],
│                   [-0.7809,  0.5063,  0.5516],
│                   [-0.3210, -0.8318, -0.4972],
│                   [-0.7809,  0.5063,  0.5516],
│                   [-0.3210, -0.8318, -0.4972],
│                   [-0.7809,  0.5063,  0.5516],
│                   [-0.3210, -0.8318, -0.4972],
│                   [-0.7809,  0.5063,  0.5516],
│                   [-0.3210, -0.8318, -0.4972]])
└── 'x' --> <FastTreeValue 0x7fb69823bd30>
    └── 'c' --> tensor([[ 0.1581, -0.6751,  0.1647,  1.9021],
                        [ 0.2731, -0.8836, -0.5057,  0.5117],
                 

In [23]:
%timeit t_cat(trees)

40.2 µs ± 509 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

78.2 µs ± 2.04 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.7809,  0.5063,  0.5516],
                [-0.3210, -0.8318, -0.4972]],
       
               [[-0.7809,  0.5063,  0.5516],
                [-0.3210, -0.8318, -0.4972]],
       
               [[-0.7809,  0.5063,  0.5516],
                [-0.3210, -0.8318, -0.4972]],
       
               [[-0.7809,  0.5063,  0.5516],
                [-0.3210, -0.8318, -0.4972]],
       
               [[-0.7809,  0.5063,  0.5516],
                [-0.3210, -0.8318, -0.4972]],
       
               [[-0.7809,  0.5063,  0.5516],
                [-0.3210, -0.8318, -0.4972]],
       
               [[-0.7809,  0.5063,  0.5516],
                [-0.3210, -0.8318, -0.4972]],
       
               [[-0.7809,  0.5063,  0.5516],
                [-0.3210, -0.8318, -0.4972]]]),
    x: Batch(
           c: tensor([[[ 0.1581, -0.6751,  0.1647,  1.9021],
                       [ 0.2731, -0.8836, -0.5057,  0.5117],
                       [-0.4602,  0.8365, -1.7004, -0.1574]],
         

In [26]:
%timeit Batch.stack(batches)

101 µs ± 2.09 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.7809,  0.5063,  0.5516],
               [-0.3210, -0.8318, -0.4972],
               [-0.7809,  0.5063,  0.5516],
               [-0.3210, -0.8318, -0.4972],
               [-0.7809,  0.5063,  0.5516],
               [-0.3210, -0.8318, -0.4972],
               [-0.7809,  0.5063,  0.5516],
               [-0.3210, -0.8318, -0.4972],
               [-0.7809,  0.5063,  0.5516],
               [-0.3210, -0.8318, -0.4972],
               [-0.7809,  0.5063,  0.5516],
               [-0.3210, -0.8318, -0.4972],
               [-0.7809,  0.5063,  0.5516],
               [-0.3210, -0.8318, -0.4972],
               [-0.7809,  0.5063,  0.5516],
               [-0.3210, -0.8318, -0.4972]]),
    x: Batch(
           c: tensor([[ 0.1581, -0.6751,  0.1647,  1.9021],
                      [ 0.2731, -0.8836, -0.5057,  0.5117],
                      [-0.4602,  0.8365, -1.7004, -0.1574],
                      [ 0.1581, -0.6751,  0.1647,  1.9021],
                      [ 0.2731, -

In [28]:
%timeit Batch.cat(batches)

181 µs ± 2.39 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

573 µs ± 11.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
